<a href="https://colab.research.google.com/github/supsi-dacd-isaac/TeachDecisionMakingUncertainty/blob/main/L3_Robust_Optimization_example_production_proble.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

AN EXAMPLE OF ROBUSTIFIED LINEAR PROGRAM
Modified by:
 1. Erick Delage (Created for ROME 14 April 2015)
 2. Erick Delage (Adapted to RSOME in November 2020)
 3. Roberto Rocchetta (January 2025)

As discussed in example 2.5 of the  [lecture notes](http://tintin.hec.ca/pages/erick.delage/MATH80624_LectureNotes.pdf) of MATH80624 at HEC Montréal.


# **Preliminaries**



1. `!pip install rsome`  
   - Installs **RSOME**, a robust stochastic optimization modeling environment.

2. `!pip install mosek`  
   - Installs **MOSEK**, a solver for optimization problems.

3. `!rm mosek.lic`  
   - Removes any existing MOSEK license file (if present).

4. `!git clone https://github.com/roberock/mosek_lic`  
   - Clones a GitHub repository containing a MOSEK license file.

5. `!cp ./mosek_lic/mosek.lic .`  
   - Copies the license file to the current directory.

6. `!rm -r ./mosek_lic`  
   - Deletes the cloned repository after extracting the license.

7. `!mkdir -p /root/mosek`  
   - Creates a directory for MOSEK in the system’s root folder.

8. `!cp ./mosek.lic /root/mosek`  
   - Copies the MOSEK license file to the required directory.

9. `#!pip install -i https://pypi.gurobi.com gurobipy`  
   - (Commented out) Would install **Gurobi**, another solver, if needed.

This setup ensures that **RSOME** and **MOSEK** are installed and properly licensed for optimization tasks. 🚀




In [1]:
!pip install rsome
!pip install mosek
!rm mosek.lic
!git clone https://github.com/roberock/mosek_lic
!cp ./mosek_lic/mosek.lic .
!rm -r ./mosek_lic
!mkdir -p /root/mosek
!cp ./mosek.lic /root/mosek
#!pip install -i https://pypi.gurobi.com gurobipy


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.8/87.8 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.7/14.7 MB 60.0 MB/s eta 0:00:00
rm: cannot remove 'mosek.lic': No such file or directory
Cloning into 'mosek_lic'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 3 (delta 0), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (3/3), done.


WARNING!!!

The following code uses a Personal Academic License file valid until **2026-jan-23**.

If you have error messages informing you about licencing issues, you may try uncommenting the installation lines for Gurobi.

Otherwise, we recommend that you obtain your own free accademic licence of either Mosek ([url](https://www.mosek.com/)) or Gurobi ([url](https://www.gurobi.com/)).

In [3]:
import numpy as np
import rsome as rso
from rsome import ro
from rsome import msk_solver as my_solver   # Import Mosek solver interface
#from rsome import grb_solver as my_solver  # Import Gurobi solver interface

For further details, see also the example 1.1.1 in the reference:

A. Ben-Tal, L. El Ghaoui, and A. Nemirovski. *Robust Optimization*. Princeton Series in Applied Mathematics, 2009.

## **Define a deterministic optimization problem**

Define a simple production problem for a company. A **deterministic optimal production model** is defined as follows:

\begin{align}
\max\limits_{\theta}\;& 6200n_1+6900n_2-(100m_1+199.90m_2+700n_1+800n_2)\\
\text{subject to}\;&  m_1 + m_2 \leq 1000  && \text{(Storage)} \\
                 & 90n_1 + 100n_2 \leq 2000  && \text{(Manpower)} \\
                 & 40n_1 + 50n_2 \leq 800  && \text{(Equipment)} \\
                 & 100m_1+199.9m_2+700n_1+800n_2 \leq 100000 &&\text{(Budget)} \\
                 & 0.01m_1+0.02m_2-0.5n_1-0.6n_2 \geq 0 &&\text{(Catalyst/Agent A)} \\
                 &  m_1\geq0, m_2\geq0, n_1 \geq 0, n_2\geq 0.
\end{align}


**Descision variables:**
- $(n_1,n_2)$: number of boxes (1000 packs/box) produced for drug type 1 and type 2
- $(m_1, m_2)$: masses of raw material type 1 and Type 2

**Model coefficients:**

- $(c_1,c_2) = (6200, 6900)~~ [\frac{CHF}{Box}]$ expected revenue per box and drug type
- $(c_3,c_4) = (100, 199.9)~~ [\frac{CHF}{kg}]$: costs of raw materials type 1 and 2
- $(c_5,c_6) = (700, 800)~~ [\frac{CHF}{Box}]$: production cost per box and drug type



## **Solve a deterministic optimization problem**

In a compact form:

\begin{align}
\min\limits_{\theta\in\mathbb{R}^{+,n_\theta}}\;& c^T \theta \\
\text{subject to}\;&  A\theta  \leq b  &&  \\
\end{align}

**Objective function:**

- Maximize $6200n_1+6900n_2-(100m_1+199.90m_2+700n_1+800n_2) $
- Minimize $ \text{Cost}(\theta, c) -\text{Revenue}(\theta, c)  = c^T \theta$

**Coefficients:**
- $c = [-(c_1-c_5),-(c_2-c_6), c_3, c_4]$

**Decision variables:**
- $\theta = [n_1, n_2, m_1, m_2]$
   

In [12]:
from scipy.optimize import linprog # This function is used to solve linear programs


# Objective function coefficients (to be maximized)
c = [-(6200-700), -(6900-800), +100, +199.90]  # max_a f(a) --> min -f(a) Negate the objective for minimization


# Inequality constraint matrix
A = [
    [0, 0, 1, 1],  # Storage
    [90, 100, 0, 0],  # ManPower
    [40, 50, 0, 0],  # Equipment
    [700, 800, 100, 199.9],   # Budget
    [0.5, 0.6, -0.01, -0.02],  # Agent A
]

# Inequality constraint vector
b = [1000,
     2000,
     800,
     100000,
     0]

def production_problem_det(c,A,b):
  bounds = [(0, None), (0, None), (0, None), (0, None)]   # decision variables must be non-negative
  return linprog(c, A_ub=A, b_ub=b, bounds=bounds) # This function is used to solve linear programs

#Run
result = production_problem_det(c,A,b)

# Determinisitc objective (net revenue)
optobj_det = -result.fun

# Print the results
# print(result)

if result.success:
  print(" ---------------- ")
  print("Optimal Solution Deterministic Problem")
  print('Deterministic objective value')

  print(f"n1 =  {result.x[0]:.2f} boxes drug 1")
  print(f"n1 =  {result.x[1]:.2f} boxes drug 2")
  print(f"m1 =  {result.x[2]:.2f} [kg] Raw material 1")
  print(f"m2 =  {result.x[3]:.2f} [kg] Raw material 2")
  print(f"Objective Function Value = {optobj_det:.2f}  CHF") #negate to get the maximized value
else:
    print("Optimization failed: ", result.message)

 ---------------- 
Optimal Solution Deterministic Problem
Deterministic objective value
n1 =  17.55 boxes drug 1
n1 =  0.00 boxes drug 2
m1 =  0.00 [kg] Raw material 1
m2 =  438.79 [kg] Raw material 2
Objective Function Value = 8819.66  CHF


## **What could go wrong?**
## **Evaluate a production damage**
We designed the production. We bought material $(m_1,m_2)$, a customer is going to buy $(n_1,n_2)$



**Unfortunately we realize that:**
* We observe an error of up to 2 % error on the estimation of the conversion rate 0.02 of this raw material 2.
* We observe an error of up to 0.5 % on the conversion rate for the raw material 1.



Hence, in the **worst-case** for the consider 'perturbation' we will only get:
* 0.0196 kg of agent A per kg of raw material 2
* 0.0095 kg of agent A per kg of raw material 1.

**We will define**

* Various uncertainty set, e.g., a box set $\mathcal{U}=\{u\in\mathbb{R}^2: \max(|u_1|,|u_2|)\leq 1  \}$
  
* Determine how much missing agent A in the worst case? $w_{\Delta_A} = \min\limits_{u\in \mathcal{U}}[\text{Produced}(u,m_1,m_2)]-\text{Consumed}(n_1,n_2)$

* Define the problem with uncertain coefficients $A(u)$
* Evaluate the worst-case profit   $\max_u c^T \theta$

In [13]:
def worst_case_production_error(m1, m2, n1, n2, error_yeld1 = 0.005, error_yeld2=0.02 ):
  """get the worst case missing Agent A (due to production errors u1, u2)"""

  # how much Agent A from the raw materials 1 and 2 ?
  produced_with_raw1 = (1-error_yeld1)*0.01*m1
  produced_with_raw2 = (1-error_yeld2)*0.02*m2

  # how much Agent A do we need to produce durgs 1 and 2 ?
  consumed_with_durg_1 = 0.5*n1
  consumed_with_durg_2 = 0.6*n2

  return -min(0, produced_with_raw1+produced_with_raw2-consumed_with_durg_1-consumed_with_durg_2)   # worst case yield [g] for agent A


det_n1, det_n2 = result.x[0],  result.x[1]
det_m1, det_m2  = result.x[2], result.x[3]
worst_case_deterministic_solution = worst_case_production_error(det_m1, det_m2,
                                                                det_n1, det_n2,
                                                                error_yeld1 = 0.005,
                                                                error_yeld2 = 0.02)


print(f'Deterministic solution: there could be up to {worst_case_deterministic_solution:.3f} [g] of missing agent A');


Deterministic solution: there could be up to 0.176 [g] of missing agent A


## **Evaluating the worst-case profit**


In [14]:
# Objective function coefficients (to be maximized)
c = [-(6200-700), -(6900-800), +100, +199.90]  # max J() --> min -J() Negated for maximization

# Inequality constraint matrix
A = [
    [0, 0, 1, 1],
    [90, 100, 0, 0],  # ManPower
    [40, 50, 0, 0],  # Equipment
    [700, 800, 100, 199.9],   # Budget
    [0.5, 0.6, -0.01*(1-0.005), -0.02*(1-0.02)],  # Catalyst
]

# Inequality constraint vector
b = [1000,
     2000,
     800,
     100000,
     0]

def production_problem_det(c,A,b):
  # decision variables are non-negative
  bounds = [(0, None), (0, None), (det_m1, det_m1), (det_m2, det_m2)] # we already ordered the raw material....so we can fix the boudns it to known solution det_m1, det_m2
  # value <= a <= value
  return linprog(c, A_ub=A, b_ub=b, bounds=bounds)  # Solve the linear program

# Run
result_wc = production_problem_det(c,A,b)

# Determinisitc revenue
optobj_det_wc = -result_wc.fun

if result_wc.success:
  print(" ---------------- ")
  print("Solution Deterministic Problem with Uncertain Yield")
  print(f"n1 =  {result_wc.x[0]:.2f} boxes drug 1")
  print(f"n1 =  {result_wc.x[1]:.2f} boxes drug 2")
  print(f"m1 =  {result_wc.x[2]:.2f} [kg] Raw material 1")
  print(f"m2 =  {result_wc.x[3]:.2f} [kg] Raw material 2")
  print(f"Objective Function Value = {optobj_det_wc:.2f}  CHF")
else:
    print("Optimization failed: ", result_wc.message)

 ---------------- 
Solution Deterministic Problem with Uncertain Yield
n1 =  17.20 boxes drug 1
n1 =  0.00 boxes drug 2
m1 =  0.00 [kg] Raw material 1
m2 =  438.79 [kg] Raw material 2
Objective Function Value = 6888.99  CHF


In [15]:
# define the paramers (this is how much material we bought....mass RawI and mass RawII)
RawI = det_m1;
RawII = det_m2;

#Create model
model = ro.Model('production_raw_fixed_worst-case_scenario')

# Define decision variables
DrugI =model.dvar()             # Define a decision variable DrugI
DrugII =model.dvar()            # Define a decision variable DrugII

#Objective to maximize the profit
model.max(6200*DrugI+6900*DrugII - (100*RawI+199.90*RawII+700*DrugI+800*DrugII))

# Manpower constraint
model.st(90*DrugI + 100*DrugII <= 2000)

# Equipment constraint
model.st(40*DrugI+50*DrugII <= 800)

# Budget constraint
model.st(100*RawI+199.9*RawII+700*DrugI+800*DrugII <= 100000)

# Constraint to have enough active agent A under worst-case scenario
model.st(0.01*(1-0.005)*RawI+0.02*(1-0.02)*RawII-0.5*DrugI-0.6*DrugII >= 0)

# Constraints that decision variables are non-negative
model.st(DrugI >= 0)                             # DrugI is non-negative
model.st(DrugII >= 0)                            # DrugII is non-negative

# solve the model
model.solve(my_solver)

wc_det = model.get()
det_sol_wcDrugI   = DrugI.get()
det_sol_wcDrugII   = DrugII.get()

print('Same resut is obtained using the RSOME package with the Mosek solver')
print('Deterministic solution: Worst-case profit is',np.round(wc_det,2), '[CHF] vs ', np.round(optobj_det,2), '[CHF] of deterministic solution')
print('Deterministic solution: The unrealized profit is',np.round(wc_det-optobj_det,2), '[CHF] 😭')

print('Deterministic solution: Due to loss in production of', int(1000*np.round(det_n1- det_sol_wcDrugI,3)), ' packs of Drug I')
print('Deterministic solution: Due to loss in production of', int(1000*np.round(det_n2 - det_sol_wcDrugII,3)), ' packs of Drug II')

Being solved by Mosek...
Solution status: Optimal
Running time: 0.0027s
Same resut is obtained using the RSOME package with the Mosek solver
Deterministic solution: Worst-case profit is 6888.99 [CHF] vs  8819.66 [CHF] of deterministic solution
Deterministic solution: The unrealized profit is -1930.67 [CHF] 😭
Deterministic solution: Due to loss in production of 351  packs of Drug I
Deterministic solution: Due to loss in production of 0  packs of Drug II


In [10]:
# Can we do better?

# Try solving the problem using a uncertainty set (box uncertainty)


## **Solving a robust model with a box uncertainty set**

Consider a robust version of the linear optimization problem we have seen before:

\begin{align}
\min\limits_{\theta\in\mathbb{R}^{+,n_\theta}}\;& c^T \theta \\
\text{subject to}\;&  A(u)\theta  \leq b,  &&   \;\forall\, u\in\mathcal{U} \\
\end{align}
where $\mathcal{U}$ is a box-shaped uncertainty set: $$\mathcal{U}:=\{u\in\mathbb{R}^2 : |u_i| \leq 1, \forall i=1, 2\}.$$

**(Robsut constraint produced Agent A):**

Revise budget constraint as follows

$$ 0.01(1-0.005u_1)m_1+0.02(1-0.02u_2)m_2-0.5n_1-0.6n_2 \geq 0   \;\forall\,(u_1,u_2)\in\mathcal{U} $$






In [16]:
#Create model
model = ro.Model('robustproduction_box')

max_error_1 = 0.005
max_error_2 = 0.02

# Define decision variables
n1 =model.dvar()              # Define a decision variable for n boxes of Drug I
n2 =model.dvar()              # Define a decision variable for n boxes of Drug II
m1 =model.dvar()              # Define a decision variable mass Raw I
m2 =model.dvar()              # Define a decision variable mass Raw II

# Declare uncertain parameters and the box uncertainty set
u = model.rvar(2)           # define 2 random variables
boxSet = (abs(u) <= 1);     # each laing within a box [-1, 1]

# Objective to maximize the profit
model.max(6200*n1+6900*n2 - (100*m1+199.90*m2+700*n1+800*n2))

# Constraints
model.st(m1 + m2 <= 1000)                         # Storage constraint
model.st(90*n1 + 100*n2 <= 2000)                  # Manpower constraint
model.st(40*n1+50*n2 <= 800)                      # Equipment constraint
model.st(100*m1+199.9*m2+700*n1+800*n2 <= 100000) # Budget constraint

# Constraints that decision variables are non-negative
model.st(n1 >= 0)           # DrugI is non-negative
model.st(n2 >= 0)           # DrugII is non-negative
model.st(m1 >= 0)           # RawI is non-negative
model.st(m2 >= 0)           # RawII is non-negative

# Constraint to have enough active agent A
model.st((0.01*(1-max_error_1*u[0])*m1+0.02*(1-max_error_2*u[1])*m2-0.5*n1-0.6*n2 >= 0).forall(boxSet))

# solve the model
model.solve(my_solver)

optobj_rob = model.get()      #get optimal objective value
rob_sol_DrugI   = n1.get()    #get optimal solution
rob_sol_DrugII  = n2.get()    #get optimal solution
rob_sol_RawI    = m1.get()    #get optimal solution
rob_sol_RawII   = m2.get()    #get optimal solution

print('Robust solution: Expected profit is', np.round(optobj_rob,2), 'CHF')
print('Robust solution: Robust Solution Raw Material I = ', np.round(rob_sol_RawI,2))
print('Robust solution: Robust Solution Raw Material II = ', np.round(rob_sol_RawII,2))
print('Robust solution: Robust Solution Drug I = ', np.round(rob_sol_DrugI,2))
print('Robust solution:  Robust Solution Drug II = ', np.round(rob_sol_DrugII,2))

print('When drifts are as small in a BOX set it is possible to avoid a potential loss of', "{0:.0f}".format((optobj_det-wc_det)/optobj_det*100),'% by giving away', "{0:.1f}".format((optobj_det-optobj_rob)/optobj_det*100),'% of profit.⚖️' )

Being solved by Mosek...
Solution status: Optimal
Running time: 0.0013s
Robust solution: Expected profit is 8294.57 CHF
Robust solution: Robust Solution Raw Material I =  877.73
Robust solution: Robust Solution Raw Material II =  0.0
Robust solution: Robust Solution Drug I =  17.47
Robust solution:  Robust Solution Drug II =  0.0
When drifts are as small in a BOX set it is possible to avoid a potential loss of 22 % by giving away 6.0 % of profit.⚖️


In [18]:
# show that we are never missing [g] of Agent A
for k in range(15):
  u = np.random.uniform(-1,1,2)  # explore different yield errors in [-1,+1]
  error_yeld1 = max_error_1*u[0]
  error_yeld2 = max_error_2*u[1]
  worst_case_robust = worst_case_production_error(m1=rob_sol_RawI,
                                                m2=rob_sol_RawII,
                                                n1=rob_sol_DrugI,
                                                n2=rob_sol_DrugII,
                                                error_yeld1 = error_yeld1,
                                                error_yeld2 = error_yeld2)

  print(f'Missing agent A for u={u} is {worst_case_robust:.3f} [g] of missing agent A');


Missing agent A for u=[ 0.09352064 -0.50163743] is 0.000 [g] of missing agent A
Missing agent A for u=[-0.07640788 -0.68185009] is 0.000 [g] of missing agent A
Missing agent A for u=[ 0.11937168 -0.69312311] is 0.000 [g] of missing agent A
Missing agent A for u=[-0.59079322 -0.94739427] is 0.000 [g] of missing agent A
Missing agent A for u=[-0.2577293   0.29435005] is 0.000 [g] of missing agent A
Missing agent A for u=[0.79185192 0.33494048] is 0.000 [g] of missing agent A
Missing agent A for u=[-0.92389903  0.79421271] is 0.000 [g] of missing agent A
Missing agent A for u=[0.58207537 0.98307422] is 0.000 [g] of missing agent A
Missing agent A for u=[-0.74423737  0.10763463] is 0.000 [g] of missing agent A
Missing agent A for u=[-0.88140806 -0.06352612] is 0.000 [g] of missing agent A
Missing agent A for u=[0.54583285 0.08022886] is 0.000 [g] of missing agent A
Missing agent A for u=[-0.51044466  0.99698051] is 0.000 [g] of missing agent A
Missing agent A for u=[-0.0304653   0.23671119

In [19]:
# WE ARE NOT GUARANTEED OUTSIDE FROM THE UNCERTAINTY SET (missing agent A)
for k in range(15):

  u = np.random.uniform(-4,4,2) # clearly, a wrong characterization of the error can lead to some issues
  error_yeld1 = max_error_1*u[0]
  error_yeld2 = max_error_2*u[1]
  worst_case_robust = worst_case_production_error(m1=rob_sol_RawI,
                                                m2=rob_sol_RawII,
                                                n1=rob_sol_DrugI,
                                                n2=rob_sol_DrugII,
                                                error_yeld1 = error_yeld1,
                                                error_yeld2 = error_yeld2)

  print(f'Missing agent A for u={u} is {worst_case_robust:.4f} [g] of missing agent A');

Missing agent A for u=[-2.03040253  0.7073723 ] is 0.0000 [g] of missing agent A
Missing agent A for u=[0.87791492 1.95721194] is 0.0000 [g] of missing agent A
Missing agent A for u=[-1.16357954  3.76703224] is 0.0000 [g] of missing agent A
Missing agent A for u=[-3.64941117  3.93534111] is 0.0000 [g] of missing agent A
Missing agent A for u=[-1.71913185 -2.87973743] is 0.0000 [g] of missing agent A
Missing agent A for u=[-1.91371989 -1.70581753] is 0.0000 [g] of missing agent A
Missing agent A for u=[-3.79831721  2.37807766] is 0.0000 [g] of missing agent A
Missing agent A for u=[-0.12518988 -1.24621658] is 0.0000 [g] of missing agent A
Missing agent A for u=[-1.4143018  -1.89749455] is 0.0000 [g] of missing agent A
Missing agent A for u=[-3.74914271 -3.9548681 ] is 0.0000 [g] of missing agent A
Missing agent A for u=[2.39893834 3.37769169] is 0.0614 [g] of missing agent A
Missing agent A for u=[0.33667145 2.7753352 ] is 0.0000 [g] of missing agent A
Missing agent A for u=[2.96216056 

## **Solve robust model with an ellipsoidal uncertainty set**

Consider the following robust optimization model with an ellipsoidal uncertainty set:

\begin{align}
\max\limits_{\theta\in\mathbb{R}^{+,n_\theta}}\;& c^T \theta \\
\text{subject to}\;&  A(u)\theta  \leq b,  &&   \;\forall\, u\in\mathcal{U} \\
\end{align}

where $\mathcal{U}:=\left\{u\in\mathbb{R}^2\;\middle|\;  ||u||_2 = \sqrt{u_1^2+u_2^2} \leq 1\right\}.$

In [20]:
#Create model
model = ro.Model('robustproduction_elipsoidal')

#implement your optimization problem with an elipsoidal uncertainty set

#  TO BE COMPLETED

Being solved by Mosek...
Solution status: Optimal
Running time: 0.0025s
Robust solution: Expected profit is 8311.76 CHF
Robust solution: Robust Solution Raw Material I =  821.12
Robust solution: Robust Solution Raw Material II =  28.31
Robust solution: Robust Solution Drug I =  17.47
Robust solution:  Robust Solution Drug II =  0.0
When drifts are as small in a BALL set it is possible to avoid a potential loss of 22 % by giving away 5.8 % of profit.⚖️


## **Solving a robust model with a budgeted uncertainty set**


Consider the following robust optimization model:
\begin{align}
\max\limits_{\theta}\;& 6200n_1+6900n_2-(100m_1+199.90m_2+700n_1+800n_2) \\
\text{subject to}\;&  m_1 + m_2 \leq 1000  && \text{(Storage)} \\
                 & 90n_1 + 100n_2 \leq 2000  && \text{(Manpower)} \\
                 & 40n_1 + 50n_2 \leq 800  && \text{(Equipment)} \\
                 & 100m_1+199.9m_2+700n_1+800n_2 \leq 100000 &&\text{(Budget)} \\
                 & 0.01(1-0.005u_1)m_1+0.02(1-0.02u_2)m_2-0.5n_1-0.6n_2 \geq 0   \;\forall\,(u_1,u_2)\in\mathcal{U} && \text{(Robsut constraint produced Agent A)} \\
                 &  m_1\geq0, m_2\geq0, n_1 \geq 0, n_2\geq 0.
\end{align}

where $\mathcal{U}$ is a box-shaped uncertainty set: $$\mathcal{U}:=\{(u_1,u_2) \in\mathbb{R}^2\;:\; \sum\limits_{i=1}^2|z_i| \leq \Gamma, |z_i| \leq 1, \forall i =1,2\}.$$

Or in a compact form:
 $$\mathcal{U}:=\{u\in\mathbb{R}^2\;:\; ||z||_{1} \leq \Gamma, ||z||_{\infty}  \leq 1\}.$$


In [23]:
#Create model
model = ro.Model('robustproduction_budgeted')

# Hint : owe can define the set as follows

# budgetedSet=(abs(u) <= 1, rso.norm(u, 1) <=Gamma)  t


#  TO BE COMPLETED .........


Being solved by Mosek...
Solution status: Optimal
Running time: 0.0014s
Robust solution: Expected profit is 8294.57 CHF
Robust solution: Robust Solution Raw Material I =  877.73
Robust solution: Robust Solution Raw Material II =  0.0
Robust solution: Robust Solution Drug I =  17.47
Robust solution:  Robust Solution Drug II =  0.0
When drifts are as small in a budgeted set it is possible to avoid a potential loss of 22 % by giving away 6.0 % of profit.⚖️


# Bounus

* Example of RO with large uncertainy on several parameters (not only on the Yield rate of Agent A)

In [25]:
#Create model
model = ro.Model('robustproduction_maxmin')

# Define decision variables
n1 =model.dvar()              # Define a decision variable for n boxes of Drug I
n2 =model.dvar()              # Define a decision variable for n boxes of Drug II
m1 =model.dvar()              # Define a decision variable mass Raw I
m2 =model.dvar()              # Define a decision variable mass Raw II


# Declare uncertain parameters
u = model.rvar(6)
BOX_U = (abs(u) <= 1 )  # BOX UNCERTAINTY ON the 6 UNCERTAIN PARAMETERS

# Objective to maximize the worst case (min) profit
model.maxmin(6200*n1*(1-0.01*u[4])+6900*n2*(1-0.01*u[5]) - (100*m1+199.90*m2+700*n1+800*n2), BOX_U)

# Constraints
model.st(m1 + m2 <= 1000)                         # Storage constraint
model.st(90*n1 + 100*n2 <= 2000)                  # Manpower constraint


# Constraints that decision variables are non-negative
model.st(n1 >= 0)           # DrugI is non-negative
model.st(n2 >= 0)           # DrugII is non-negative
model.st(m1 >= 0)           # RawI is non-negative
model.st(m2 >= 0)           # RawII is non-negative


# Constraint to have enough active agent A
model.st((40*n1+50*n2 - 800*(1-0.005*u[2]) <= 0).forall(BOX_U))                      # Equipment constraint
model.st((100*m1+199.9*m2+700*n1+800*n2 -100000*(1-0.2*u[3])<= 0).forall(BOX_U)) # Budget constraint
model.st((0.01*(1+0.005*u[0])*m1+0.02*(1+0.02*u[1])*m2-0.5*n1-0.6*n2 >= 0).forall(BOX_U))

# solve the model
model.solve(my_solver)

optobj_rob = model.get()      #get optimal objective value
rob_sol_DrugI   = n1.get()    #get optimal solution
rob_sol_DrugII  = n2.get()    #get optimal solution
rob_sol_RawI    = m1.get()    #get optimal solution
rob_sol_RawII   = m2.get()    #get optimal solution

print('Robust solution: Expected profit is', np.round(optobj_rob,2), 'CHF')
print('Robust solution: Robust Solution Raw Material I = ', np.round(rob_sol_RawI,2))
print('Robust solution: Robust Solution Raw Material II = ', np.round(rob_sol_RawII,2))
print('Robust solution: Robust Solution Drug I = ', np.round(rob_sol_DrugI,2))
print('Robust solution:  Robust Solution Drug II = ', np.round(rob_sol_DrugII,2))
print('When drifts are as small in a budgeted set it is possible to avoid a potential loss of', "{0:.0f}".format((optobj_det-wc_det)/optobj_det*100),'% by giving away', "{0:.1f}".format((optobj_det-optobj_rob)/optobj_det*100),'% of profit.⚖️' )


Being solved by Mosek...
Solution status: Optimal
Running time: 0.0024s
Robust solution: Expected profit is 5769.3 CHF
Robust solution: Robust Solution Raw Material I =  702.19
Robust solution: Robust Solution Raw Material II =  0.0
Robust solution: Robust Solution Drug I =  13.97
Robust solution:  Robust Solution Drug II =  0.0
When drifts are as small in a budgeted set it is possible to avoid a potential loss of 22 % by giving away 34.6 % of profit.⚖️
